## Load an LTSF-Linear Model

Train the model through 'electricity.sh' bash script.
Load an .pth file from the checkpoints folder.

In [15]:
import torch
from torchviz import make_dot
import pandas as pd
import sys
import os
sys.path.append("..")
from models.DLinear import Model as DLinear
from models.Linear import Model as Linear
from models.NLinear import Model as NLinear

class Configs:
    def __init__(self, seq_len, pred_len, enc_in):
      self.seq_len = seq_len
      self.pred_len = pred_len
      self.enc_in = enc_in
      self.individual = True

model = 'Linear'
enc_in = 50
seq_len = 192
pred_len = 24

configs = Configs(seq_len, pred_len, enc_in)
match model:
  case 'Linear':
    circuit = Linear(configs)
  case 'DLinear':
    circuit = DLinear(configs)
  case 'NLinear':
    circuit = NLinear(configs)

basepath = '../checkpoints/Electricity_192_24_Linear_custom_ftM_sl192_ll48_pl24_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_0'
newpath = basepath + '_{}'.format(enc_in)
if os.path.isdir(basepath):
  os.rename(basepath, newpath)

check_point_model = os.path.join(newpath, 'checkpoint.pth')

state_dict = torch.load(check_point_model)
circuit.load_state_dict(state_dict)

total_params = sum(
	param.numel() for param in circuit.parameters()
)

print(circuit, 'with {} parameters'.format(total_params))

df = pd.read_csv('../dataset/electricity.csv')
df = df.iloc[:, :enc_in+1]
# Load the last seq_len entries data as input and converts to tensor
x = torch.tensor(df[-seq_len:].drop(labels=['date'], axis=1).values, requires_grad=True).resize(1, seq_len, len(df.columns) - 1).float()
# Flips the neural net into inference mode
circuit.eval()

y = circuit(x)
print(y)

Model(
  (Linear): ModuleList(
    (0-49): 50 x Linear(in_features=192, out_features=24, bias=True)
  )
) with 231600 parameters


/mnt/zkpet/venv/lib/python3.11/site-packages/torch/_tensor.py:836: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


tensor([[[ 10.4050,  80.6632,   7.6226,  ..., 197.5994,  74.7061, 612.8615],
         [ 10.5257,  76.0636,   7.9764,  ..., 182.8274,  49.7463, 515.0209],
         [ 10.4413,  72.1204,   8.0914,  ..., 172.0903,  36.8496, 461.4332],
         ...,
         [ 11.3484,  88.4742,   7.7014,  ..., 236.9456,  61.0505, 713.3558],
         [ 10.4254,  85.9661,   7.8380,  ..., 221.0525,  82.6110, 641.0234],
         [ 10.3827,  82.7221,   8.1247,  ..., 209.1553,  67.1848, 576.4092]]],
       grad_fn=<CopySlices>)


## ZK Inference

#### Define Files Path

In [16]:
import tracemalloc
import os
import json
from timeit import default_timer as timer

model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.ezkl')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')
srs_path = os.path.join('kzg.srs')
witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')
proof_path = os.path.join('test.pf')
sol_code_path = os.path.join('verify.sol')
abi_path = os.path.join('verify.abi')

#### Convert Model to ONNX

In [17]:
# Model was trained by 'electricity.sh' and stored into the checkpoint state 'checkpoint.pth'.
# Now we need to export the onnx file from this state file with model inputs.

# Export the model
torch.onnx.export(circuit,               # model being run
                  x,                   # model input (or a tuple for multiple inputs)
                  model_path,            # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=15,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

# Serialize data into file:
json.dump( data, open(data_path, 'w' ))

input_size = os.stat(data_path).st_size / 1024
onnx_size = os.stat(model_path).st_size / 1024
print("Input.json size: {}KB".format(input_size))
print("network.onnx size: {}KB".format(onnx_size))

Input.json size: 64.1669921875KB
network.onnx size: 1191.4677734375KB


### Setting circuit parameters

In [18]:
import ezkl

run_args = ezkl.PyRunArgs()
run_args.input_visibility = "private"
run_args.output_visibility = "public"
run_args.param_visibility = "fixed"
run_args.variables = [("batch_size", 1)]
# run_args.logrows = 20

try:
    table_string = ezkl.table(model_path, run_args)
#     print(table_string)
except Exception as e:
    print(f"An error occurred: {e}")

In [19]:
res = ezkl.gen_settings(model_path, settings_path, py_run_args=run_args)
assert res == True

res = ezkl.calibrate_settings(data_path, model_path, settings_path, "resources")
assert res == True



 <------------- Numerical Fidelity Report (input_scale: 13, param_scale: 13, scale_input_multiplier: 10) ------------->

+---------------+---------------+-----------+------------+----------------+------------------+---------------+----------------+--------------------+--------------------+------------------------+
| mean_error    | median_error  | max_error | min_error  | mean_abs_error | median_abs_error | max_abs_error | min_abs_error  | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+---------------+---------------+-----------+------------+----------------+------------------+---------------+----------------+--------------------+--------------------+------------------------+
| -0.0019971626 | 0.00005340576 | 6.5859375 | -5.2851563 | 0.2263001      | 0.00005340576    | 6.5859375     | 0.000030517578 | 0.36669162         | -0.0000028767922   | 0.0005136585           |
+---------------+---------------+-----------+------------+----------------+------------------+---

### Compiling the Model

In [20]:
tracemalloc.start()
start = timer()

res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

end = timer()
print("Compilation time used: {} seconds.".format(end - start))

curr_size, peak_size = tracemalloc.get_traced_memory()
print("Current memory size {} KB, peak memory size {} KB".format(round(curr_size / 1024, 3),
                                                              round(peak_size / 1024, 3)))

Compilation time used: 0.7594859087839723 seconds.
Current memory size 14085.989 KB, peak memory size 136159.106 KB


### Creating the circuit

In [21]:
# get public srs from kzg ceremony, saved to srs path.
try:
    os.remove(srs_path)
except OSError:
    pass

res = ezkl.get_srs(srs_path=srs_path, settings_path=settings_path)

In [22]:
# Setup is performed by the application developer, 
# who then deploys the resulting artifacts to production.

# setup the circuit and make sure the keys are generated afterwards. 
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        srs_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)

: 

### Making a proof

In [ ]:
# # Prove, invoked with ezkl prove at the cli or ezkl.prove() in Python, is called by the prover, often on the client.
# import tracemalloc
# import numpy as np

# tracemalloc.start()
# current_memories = []
# peak_memories = []
# prove_times = []
# for i in range(10): 
#     start = timer()
#     res = ezkl.gen_witness(
#             data_path, 
#             compiled_model_path, 
#             witness_path
#           )
#     assert os.path.isfile(witness_path)

#     # GENERATE A PROOF

#     res = ezkl.prove(
#             witness_path,
#             compiled_model_path,
#             pk_path,
#             proof_path,
#             "single",
#             srs_path,
#         )
#     assert os.path.isfile(proof_path)
#     end = timer()
#     prove_time = end - start

#     current, peak = tracemalloc.get_traced_memory()
#     current_memories.append(current/(1024*1024))
#     peak_memories.append(peak/(1024*1024))
#     prove_times.append(prove_time)
#     tracemalloc.reset_peak()
#     # tracemalloc.clear_traces()
#     del current, peak, start, end
# print('Average current memory [MB]: {}, average peak memory [MB]: {} +/- {}'.format(
#       round(np.mean(current_memories), 4), round(np.mean(peak_memories), 4), 
#       round(np.std(peak_memories), 4))
# )

In [ ]:
res = ezkl.gen_witness(
        data_path, 
        compiled_model_path, 
        witness_path
      )
assert os.path.isfile(witness_path)

In [36]:
# Prove, invoked with ezkl prove at the cli or ezkl.prove() in Python, is called by the prover, often on the client.
import tracemalloc
import numpy as np

tracemalloc.start()
current_memories = []
peak_memories = []
prove_times = []

# GENERATE A PROOF

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        "single",
        srs_path,
    )
assert os.path.isfile(proof_path)
end = timer()
prove_time = end - start

current, peak = tracemalloc.get_traced_memory()
current_memories.append(current/(1024*1024))
peak_memories.append(peak/(1024*1024))
prove_times.append(prove_time)
tracemalloc.reset_peak()
    # tracemalloc.clear_traces()
    # del current, peak, start, end
print('Average current memory [MB]: {}, average peak memory [MB]: {} +/- {}'.format(
      round(np.mean(current_memories), 4), round(np.mean(peak_memories), 4), 
      round(np.std(peak_memories), 4))
)

Average current memory [MB]: 13.6138, average peak memory [MB]: 136.2881 +/- 0.0


### Verify

#### VERIFY off-chain

In [37]:
start = timer()
res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        srs_path,
    )

assert res == True
print("verified")
end = timer()
print("time used: {} seconds".format(end - start))

verified
time used: 0.8323131599463522 seconds


#### VERIFY on-chain

In [38]:
# Create verifier contract
res = ezkl.create_evm_verifier(
        vk_path=vk_path,
        srs_path=srs_path,
        settings_path=settings_path,
        sol_code_path=sol_code_path,
        abi_path=abi_path,
    )
verifier_size = os.stat(sol_code_path).st_size / 1024
print("{} size: {}KB".format(sol_code_path, verifier_size))

verify.sol size: 84.9794921875KB


In [39]:
# 1. install anvil if you haven't already
# cargo install --git https://github.com/foundry-rs/foundry --profile local --locked anvil
# 2. spin up a local EVM through anvil in a separate terminal 
# anvil -p 3030

# Deploy the verifier contract onchain
sol_code_path = os.path.join("verify.sol")
address_path = os.path.join('contractAddr.txt')
# assuming anvil is running
res = ezkl.deploy_evm(
    address_path,
    sol_code_path,
    'http://127.0.0.1:3030'
)

assert res == True

In [40]:
# Verify proof onchain

with open(address_path, 'r') as f:
  addr = f.readline()

res = ezkl.verify_evm(
    proof_path=proof_path,
    addr_verifier=addr,
    rpc_url='http://127.0.0.1:3030'
)

assert res == True

## Stats

In [41]:
from subprocess import Popen, PIPE

input_size = round(os.stat(data_path).st_size / 1024, 3)
onnx_size = round(os.stat(model_path).st_size / 1024, 3)
print("{} size: {}KB".format(data_path, input_size))
print("{} size: {}KB".format(model_path, onnx_size))
proof_size = round(os.stat(proof_path).st_size / 1024, 3)
print("{} size: {}KB".format(proof_path, proof_size))
prove_time = round(np.mean(prove_times), 3)
print("prove time used: {} seconds".format(prove_time))
prove_mem = round(np.mean(peak_memories), 3)
print("prove memory used: {} MB".format(prove_mem))

verifier_size = round(os.stat(sol_code_path).st_size / 1024, 3)
print("{} size: {}KB".format(sol_code_path, verifier_size))

p = Popen(["solc", "--bin", "--optimize", "verify.sol"], stdin=PIPE, stdout=PIPE, stderr=PIPE)
output, err = p.communicate(b"input data that is passed to subprocess' stdin")
verifier_code_size = round(sys.getsizeof((output.split(b'\n')[3])) / 1000, 3)
print("Verifier bytecode size: {}KB".format(verifier_code_size))

input.json size: 37.968KB
network.onnx size: 713.995KB
test.pf size: 132.24KB
prove time used: 188.903 seconds
prove memory used: 136.288 MB
verify.sol size: 84.979KB
Verifier bytecode size: 21.299KB


In [42]:
colums = ['Parameters', 'Input Size', 'ONNX Size', 'Proof Size',
          'Prove Time', 'Prove Mem', 'Verifier Size', 'Bytecode Size']
stats = [[total_params], [input_size], [onnx_size], [proof_size],
        [prove_time], [prove_mem], [verifier_size], [verifier_code_size]]
data = dict(zip(colums, stats))
index = [seq_len]
df = pd.DataFrame(data=data, index=index)
output_path='zkml_perf.csv'
df.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
df

,Parameters,Input Size,ONNX Size,Proof Size,Prove Time,Prove Mem,Verifier Size,Bytecode Size
192,138960,37.968,713.995,132.24,188.903,136.288,84.979,21.299


In [43]:
# 20 nodes: downsizing params to 18 logrows 4625160
max_mem = 4631932 / 1024

In [1]:
4631932 / 1024

4523.37109375

In [44]:
elapsed_time = 97

In [45]:
# 10 nodes
max_mem = 2311880 / 1024

In [2]:
2311880 / 1024

2257.6953125

In [46]:
elapsed_time = 47

In [3]:
9214232 / 1024

8998.2734375

- 30: Time: 3:35.66 max_mem: 9214232 / 1024
- 40: Time: 5:09.51 Max_mem 14993000 / 1024

In [13]:
14993000 / 1024

14641.6015625

In [14]:
import subprocess

# /usr/bin/time -v ezkl prove --witness witness.json -M network.ezkl --proof-path test.pf --pk-path test.pk --srs-path=kzg.srs

result = subprocess.run(['/usr/bin/time', '-v', 'ezkl', 'prove', '--witness', witness_path, '-M', compiled_model_path, '--proof-path', proof_path, '--pk-path', pk_path, '--srs-path', srs_path])
result.stdout

[*] [2024-04-06 20:28:54, ezkl] - 
 |  
 |          ███████╗███████╗██╗  ██╗██╗
 |          ██╔════╝╚══███╔╝██║ ██╔╝██║
 |          █████╗    ███╔╝ █████╔╝ ██║
 |          ██╔══╝   ███╔╝  ██╔═██╗ ██║
 |          ███████╗███████╗██║  ██╗███████╗
 |          ╚══════╝╚══════╝╚═╝  ╚═╝╚══════╝
 |  
 |          -----------------------------------------------------------
 |          Easy Zero Knowledge for the Lyrical.
 |          -----------------------------------------------------------
 |  
 |          
[*] [2024-04-06 20:28:54, ezkl::pfsys] - loading proving key from "test.pk"
[*] [2024-04-06 20:30:11, ezkl::pfsys] - done loading proving key ✅
[*] [2024-04-06 20:30:11, ezkl::pfsys::srs] - loading srs from "kzg.srs"
[*] [2024-04-06 20:30:13, ezkl::execute] - downsizing params to 20 logrows
[*] [2024-04-06 20:30:13, ezkl::pfsys] - proof started...
[*] [2024-04-06 20:30:15, ezkl::graph::model] - model layout...


Command terminated by signal 9
	Command being timed: "ezkl prove --witness witness.json -M network.ezkl --proof-path test.pf --pk-path test.pk --srs-path kzg.srs"
	User time (seconds): 755.69
	System time (seconds): 72.55
	Percent of CPU this job got: 224%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 6:09.02
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 14564584
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 13402
	Minor (reclaiming a frame) page faults: 6733350
	Voluntary context switches: 119508
	Involuntary context switches: 455555
	Swaps: 0
	File system inputs: 19567880
	File system outputs: 0
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0
